In [ ]:
# match_details
#    a row for every players performance in a match

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast

spark = SparkSession.builder.appName("Homework").getOrCreate()

24/12/09 15:53:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [3]:
# import match_detail
match_details_bucketed = spark.read.option("header", "true") \
                .option("inferSchema", "true") \
                .csv("/home/iceberg/data/match_details.csv")

# import matches
matches_bucketed = spark.read.option("header", "true") \
                .option("inferSchema", "true") \
                .csv("/home/iceberg/data/matches.csv")

# import medals_matches_players
medals_matches_players_bucketed = spark.read.option("header", "true") \
                .option("inferSchema", "true") \
                .csv("/home/iceberg/data/medals_matches_players.csv")

# import medals
medals_bucketed = spark.read.option("header", "true") \
                .option("inferSchema", "true") \
                .csv("/home/iceberg/data/medals.csv") 

In [ ]:
spark.sql("""DROP TABLE IF EXISTS bootcamp.matches_bucketed""")
spark.sql("""DROP TABLE IF EXISTS bootcamp.matches_bucketed""")
spark.sql("""DROP TABLE IF EXISTS bootcamp.matches_bucketed""")
spark.sql("""DROP TABLE IF EXISTS bootcamp.matches_bucketed""")

In [4]:
# disabling broadcast join threshold
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [7]:
matches_bucketedDDL = """CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
                         match_id STRING,
                         is_team_game BOOLEAN,
                         playlist_id STRING,
                         completion_date TIMESTAMP
                     )
                     USING iceberg
                     PARTITIONED BY (completion_date, bucket(16, match_id));
                     """
spark.sql(matches_bucketedDDL)

matches_bucketed.select("match_id", "is_team_game", "playlist_id", "completion_date") \
   .write.mode("append") \
   .saveAsTable("bootcamp.matches_bucketed")

24/12/09 16:06:09 ERROR Utils: Aborting task                        (0 + 1) / 1]
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.io.ByteArrayOutputStream.<init>(ByteArrayOutputStream.java:79)
	at org.apache.iceberg.shaded.org.apache.parquet.hadoop.CodecFactory$HeapBytesCompressor.<init>(CodecFactory.java:158)
	at org.apache.iceberg.shaded.org.apache.parquet.hadoop.CodecFactory.createCompressor(CodecFactory.java:219)
	at org.apache.iceberg.shaded.org.apache.parquet.hadoop.CodecFactory.getCompressor(CodecFactory.java:202)
	at org.apache.iceberg.parquet.ParquetWriter.<init>(ParquetWriter.java:90)
	at org.apache.iceberg.parquet.Parquet$WriteBuilder.build(Parquet.java:352)
	at org.apache.iceberg.parquet.Parquet$DataWriteBuilder.build(Parquet.java:734)
	at org.apache.iceberg.data.BaseFileWriterFactory.newDataWriter(BaseFileWriterFactory.java:131)
	at org.apache.iceberg.io.RollingDataWriter.newWriter(RollingDataWriter.java:52)
	at org.apache.iceberg.io.RollingDataWriter.newWrit

Py4JError: py4j.reflection.TypeUtil.isInstanceOf does not exist in the JVM

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [ ]:
match_details.createOrReplaceTempView("match_details")

data = spark.sql("SELECT * FROM match_details")
data.show(truncate=False)
data.count()

In [ ]:
df.createOrReplaceTempView("filtered_df")

data = spark.sql("""
                    WITH deduped AS (
                        SELECT
                            player_gamertag, match_id, team_id,
                            ROW_NUMBER() OVER (PARTITION BY player_gamertag, match_id, team_id ORDER BY match_id) AS row_num
                        FROM filtered_df
                        )
                        SELECT * FROM deduped
                        WHERE row_num = 1
                    """)
data.show(truncate=False)
# count rows
data.count()